# Convert `.tml` Varian print template file to `.json`

In [222]:
from pathlib import Path
import re


In [223]:
base_path = Path.cwd()
file_name = 'AllVariables.tml'
file = base_path / file_name

output_file_name = 'AllVariablesJson.tml'
output_file = base_path / output_file_name

In [224]:
raw_template = file.read_text(encoding='utf_8_sig')

In [225]:
#print(raw_template)
#raw_template

In [226]:
regex_patterns = [
    # Remove lines containing formatting instructions
    (r'^[ ]*Description=.+$', ''),
    (r'^[ ]*PaperSize=.+$', ''),
    (r'^[ ]*PaperOrientation=.+$', ''),
    (r'^[ ]*TextRight=.+$', '',),
    (r'^[ ]*Height=.+$', ''),
    (r'^[ ]*PageBreak=.+$', ''),
    (r'^[ ]*Section=.+$', ''),
    (r'^[ ]*Font=.+$', ''),
    (r'^[ ]*HorizontalLine=.+$', ''),
    (r'^[ ]*Margin=.+$', ''),
    (r'^[ ]*RelativeMove=.+$', ''),
    (r'^[ ]*RelativeMoveText=.+$', ''),
    (r'^[ ]*Move=.+$', ''),
    (r'^[ ]*Color=.+$', ''),
    # Remove any DVH Graph
    (r'DVHGraph=.+$', ''),
    # Convert lines with the format: Text=*<GROUP NAME>* VARIABLES
    # to Json objects that contain objects:
    # },                       # Ends previous object group
    # "*<GROUP NAME>*": {      # Starts a new group 
    (r'Text=([A-Za-z ]+) (VARIABLES|Variables)([\r\n]+)', r'},\3"\1": {\3'),
    # convert lines with the format: Text=$$ *<VariableName>* : $*<VariableName>*
    # to Json objects: "*<VariableName>*":, $*<VariableName>*,
    (r'Text=[$]{2}[ ]*([A-Za-z_0-9]+)[ ]*:[ ]*([$][A-Za-z_0-9]+)', r'"\1": "\2 ",'),
    #for lines with the format: 
    # Text=$$ *<VariableName1>* $$ *<VariableName2>*: $*<VariableName>*
    # Drop the second variable name
    (r'Text=[$]{2}[ ]*([A-Za-z_0-9]+)[ ]*[$]{2}[ ]*([A-Za-z_0-9]+)[ ]*:[ ]*([$][A-Za-z_0-9$#]+)', r'"\1": "\3 ",'),
    # Convert lines within a loop to an array:
    # convert lines with the format: Loop=*<VariableName>*
    # to a Json array of objects:
    # *<indent>*"*<VariableName>*": [{
    # embed original Loop command from tml inside Json array structure.     
    # Loop=*<VariableName>*   
    (r'([ ]*)Loop=([A-Za-z_0-9]+)([\r\n]+)', r'\1"\2": [{\3\1Loop=\2\3\1'),
    # Add end-of array Json syntax after an EndLoop  command:
    # convert lines with the format: Loop=*<VariableName>*
    # to a Json array of objects:
    # *<indent>*}
    # *<indent>*EndLoop
    # *<indent>*],
    (r'([ ]*)EndLoop([\r\n]+)', r'\1},{\2\1EndLoop\2\1}],\2'),
    # Remove comment lines containing 'Text='
    (r'^[ ]*Text=[^\r\n]*$', r''),    
    # Remove comments (everything on a line after '#', 
    # except when '#' is preceded by '$')
    (r'[#](?<!$#)[^\r\n]*$', ''),
    # Remove contents of lines that contain only spaces.
    (r'^[ ]+$', ''),
    # Remove commas with no following object
    (r',(\s*[}\]])', r'\1'),
    ]

**Object levels incorrect***

`JSONDecodeError: Expecting ',' delimiter: line 1027 column 14 (char 27962)`

This template text:
```
Text=Field Variables
Loop=Fields
  Text=$$ FieldName : $FieldName
  Text=$$ FieldId : $FieldId
...
  Text=$$ ApplicatorId : $ApplicatorId
  Text=
  Text=Calculation Variables
  Text=$$ FieldCalculationTimestamp : $FieldCalculationTimestamp
...
  Text=$$ PostProcessingExecuted : $PostProcessingExecuted
  Text=MLC Variables
  Text=$$ FieldMLCs : $FieldMLCs
...
  Text=$$ MLCPlanSegments : $MLCPlanSegments
```

 is converted to:
 ```
},
"Field": {
"Fields": [{
```
Need something like: `"Parameters": {` here
```
  "FieldName": "• KV ANT •",
  "FieldId": "• KV ANT •",
...
  "ApplicatorId": "-"
  },
"Calculation": {
  "FieldCalculationTimestamp": "Monday, February 6, 2023 2:15:36 PM",
...
  "PostProcessingExecuted": ""
  },
"MLC": {
  "FieldMLCs": "0",
...
  "MLCPlanSegments": "-"
  },
  ...
  "Carriage Group": {
  "CarriageGroups": [{
  }]
}
},{
```
**Change last `},{` to `},`**

In [227]:
mod_str = raw_template
for ptrn, repl in regex_patterns:
    mod_str, rep_num = re.subn(ptrn, repl, mod_str, flags=re.MULTILINE)
    print(f'Replaced {rep_num} instances of {ptrn}')
    
    

# Remove blank lines
while '\n\n' in mod_str:
    mod_str = mod_str.replace('\n\n', '\n')
# Replace the first }, with {
mod_str = mod_str.replace('},', '{', 1)
# Add '}' to the end of the text
mod_str = mod_str + '}'


Replaced 1 instances of ^[ ]*Description=.+$
Replaced 1 instances of ^[ ]*PaperSize=.+$
Replaced 1 instances of ^[ ]*PaperOrientation=.+$
Replaced 1 instances of ^[ ]*TextRight=.+$
Replaced 2 instances of ^[ ]*Height=.+$
Replaced 1 instances of ^[ ]*PageBreak=.+$
Replaced 4 instances of ^[ ]*Section=.+$
Replaced 58 instances of ^[ ]*Font=.+$
Replaced 20 instances of ^[ ]*HorizontalLine=.+$
Replaced 72 instances of ^[ ]*Margin=.+$
Replaced 35 instances of ^[ ]*RelativeMove=.+$
Replaced 1 instances of ^[ ]*RelativeMoveText=.+$
Replaced 76 instances of ^[ ]*Move=.+$
Replaced 1 instances of ^[ ]*Color=.+$
Replaced 2 instances of DVHGraph=.+$
Replaced 28 instances of Text=([A-Za-z ]+) (VARIABLES|Variables)([\r\n]+)
Replaced 424 instances of Text=[$]{2}[ ]*([A-Za-z_0-9]+)[ ]*:[ ]*([$][A-Za-z_0-9]+)
Replaced 6 instances of Text=[$]{2}[ ]*([A-Za-z_0-9]+)[ ]*[$]{2}[ ]*([A-Za-z_0-9]+)[ ]*:[ ]*([$][A-Za-z_0-9$#]+)
Replaced 20 instances of ([ ]*)Loop=([A-Za-z_0-9]+)([\r\n]+)
Replaced 20 instances 

In [229]:
#mod_str
#print(mod_str)

In [230]:
command_text = [
    'Loop',
    'EndLoop',
    'If',
    'EndIf',
    'Text=',
    '#'
    ]
prt_tmpl_list = []
for line in mod_str.splitlines():
    if not any(cm in line for cm in command_text):
        line = 'Text=' + line
    prt_tmpl_list.append(line)


In [231]:
final_json_prt_tmpl = '\n'.join(prt_tmpl_list)
# Add 'Section=Form' to the beginning  of the text
final_json_prt_tmpl = 'Section=Form\n\n' + final_json_prt_tmpl

In [232]:
output_file.write_text(final_json_prt_tmpl, encoding='utf_8_sig')

23874

In [201]:
#print(final_json_prt_tmpl)

# Reading Json File

In [202]:
import json

***`JSONDecodeError: Expecting property name enclosed in double quotes: line 15 column 1 (char 343)`***

 
{
"PLAN SUM": {
"PlanSumId": "-",
"PlanSumName": "-",
"PlanSumCourseId": "-",
"SumPlans": [{
  "PlanIdOfSum": "BRAI-FSRT",
  "PlanNameOfSum": "BRAI-FSRT",
  "DosePerFrac_ForSum": "900.0 cGy",
  "NbrOfFrac_ForSum": "3",
  "TotalDose_ForSum": "2700.0 cGy",
  "TotalDoseAtPrimary_ForSum": "2700.0 cGy",
  "DosePerFraction_ForSum": "900.0 cGy",
},{
}],
},


In [238]:
test_file = Path(r'\\dkphysicspv1\e$\Gregs_Work\Temp\Plan Checking Temp\JasonTestFile.txt')
json_text = test_file.read_text(encoding='utf_8_sig')
json.loads(json_text)

JSONDecodeError: Expecting ',' delimiter: line 1027 column 14 (char 27962)

***Messages about calculation logs***
```
  Text=Calculation logs can be printed line by line using any of the following loops:
  Text= - CalculationWarnings
  Text= - CalculationErrors
  Text= - CalculationInfos
  Text= - CalculationNotes
  Text= - LmcCalculationErrors
  Text= - LmcCalculationWarnings
  Text= - LmcCalculationInfos
  Text= - LmcCalculationNotes
  Text= - CompCalculationErrors
  Text= - CompCalculationWarnings
  Text= - CompCalculationInfos
  Text= - CompCalculationNotes
  Text= - PortalDoseCalculationErrors
  Text= - PortalDoseCalculationWarnings
  Text= - PortalDoseCalculationInfos
  Text= - PortalDoseCalculationNotes
  Text= - IMRTOptimizationErrors
  Text= - IMRTOptimizationWarnings
  Text= - IMRTOptimizationInfos
  Text= - IMRTOptimizationNotes
  Text= - VMATOptimizationErrors
  Text= - VMATOptimizationWarnings
  Text= - VMATOptimizationInfos
  Text= - VMATOptimizationNotes
  Text= - BeamAngleOptimizationErrors
  Text= - BeamAngleOptimizationWarnings
  Text= - BeamAngleOptimizationInfos
  Text= - BeamAngleOptimizationNotes
  Text= - BLCalculationErrors
  Text= - BLCalculationWarnings
  Text= - BLCalculationInfos
  Text= - BLCalculationNotes
  Text= - BeamLineModifiers
  Text= - BLOptimizerCalculationErrors
  Text= - BLOptimizerCalculationWarnings
  Text= - BLOptimizerCalculationInfos
  Text= - BLOptimizerCalculationNotes
  Text= - PostProcessingCalculationErrors
  Text= - PostProcessingCalculationWarnings
  Text= - PostProcessingCalculationInfos
  Text= - PostProcessingCalculationNotes
  Text=Inside the loop the variable
  Text=  $$ LogLine
  Text=prints out the current line.
  Text=The entire dose calculation log for the active field is printed using:
  Text=$$ FieldLog :
  Text=$FieldLog
  ```